# Entropix-local - Research Preview - Llama 3.2 1B/3B

<a href="https://colab.research.google.com/github/xjdr-alt/entropix-local/blob/research/entropix_local_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Select model size

In [ ]:
SELECTED_MODEL_SIZE = "1B" # Selections: 1B, 3B
TOKEN = '' # Your HuggingFace token for gated model access

# Entropix Config

In [ ]:
# Dynamic module config for Entropix Sampler

class EntropixConfig:
    def __init__(self):
        # Sampler state toggles
        ## Low Entropy, Low Varentropy: "flowing with unspoken intent"
        self.state_flowing = False
        ## High Entropy, Low Varentropy: "treading carefully, asking clarifying questions"
        self.state_treading = False
        ## Low Entropy, High Varentropy: "exploring forks in the path"
        self.state_exploring = False
        ## High Entropy, High Varentropy: "resampling in the mist"
        self.state_resampling = False

        # Extra sampler state toggles for advanced testing
        self.state_extras_agreement = False
        self.state_extras_interaction_strength = False

# Installs and initial imports

In [ ]:
!pip install tiktoken
!pip install blobfile

# Config

In [ ]:
from typing import NamedTuple, Dict, Union, Optional, List
from pathlib import Path
from functools import partial
from dataclasses import dataclass
import torch
import torch.nn.functional as F

@dataclass
class ModelConfig:
    """Configuration class for model parameters."""
    dim: int
    n_layers: int
    n_heads: int
    n_kv_heads: int
    vocab_size: int
    norm_eps: float
    rope_theta: float
    use_scaled_rope: bool
    max_seq_len: int
    model_size: str

# Define configurations for different models and model sizes
MODEL_CONFIGS = {
    "3B": ModelConfig(
        dim=3072,
        n_layers=28,
        n_heads=24,
        n_kv_heads=8,
        vocab_size=128256,
        norm_eps=1e-05,
        rope_theta=500000.0,
        use_scaled_rope=True,
        max_seq_len=8192,
        model_size="3B"
    ),
    "1B": ModelConfig(
        dim=2048,
        n_layers=16,
        n_heads=32,
        n_kv_heads=8,
        vocab_size=128256,
        norm_eps=1e-05,
        rope_theta=500000.0,
        use_scaled_rope=True,
        max_seq_len=4096,
        model_size="1B"
    )
}

MODEL_IDS = {
    "3B": "meta-llama/Llama-3.2-3B-Instruct",
    "1B": "meta-llama/Llama-3.2-1B-Instruct"
}

class ModelParams(NamedTuple):
    n_layers: int
    n_local_heads: int
    n_local_kv_heads: int
    head_dim: int
    max_seq_len: int
    rope_theta: float
    use_scaled_rope: bool

def get_model_params(config: ModelConfig) -> ModelParams:
    """Create ModelParams from config."""
    return ModelParams(
        n_layers=config.n_layers,
        n_local_heads=config.n_heads,
        n_local_kv_heads=config.n_kv_heads,
        head_dim=config.dim // config.n_heads,
        max_seq_len=config.max_seq_len,
        rope_theta=config.rope_theta,
        use_scaled_rope=config.use_scaled_rope
    )

# Download Weights

In [ ]:
import os
import torch
import ml_dtypes
import jax.numpy as jnp
import jax
from pathlib import Path

from transformers import AutoModelForCausalLM
from unittest.mock import patch
from transformers.dynamic_module_utils import get_imports

In [ ]:
def translate_key(in_key: str):
    out_key = in_key.replace('.weight', '')
    if out_key.startswith('model.'):
        out_key = out_key.replace('model.', '')
        if out_key.endswith('input_layernorm'):
            out_key = out_key.replace('input_layernorm', 'attention_norm')
        elif out_key.endswith('mlp.down_proj'):
            out_key = out_key.replace('mlp.down_proj', 'feed_forward.w2')
        elif out_key.endswith('mlp.gate_proj'):
            out_key = out_key.replace('mlp.gate_proj', 'feed_forward.w1')
        elif out_key.endswith('mlp.up_proj'):
            out_key = out_key.replace('mlp.up_proj', 'feed_forward.w3')
        elif out_key.endswith('post_attention_layernorm'):
            out_key = out_key.replace('post_attention_layernorm', 'ffn_norm')
        elif out_key.endswith('self_attn.k_proj'):
            out_key = out_key.replace('self_attn.k_proj', 'attention.wk')
        elif out_key.endswith('self_attn.o_proj'):
            out_key = out_key.replace('self_attn.o_proj', 'attention.wo')
        elif out_key.endswith('self_attn.q_proj'):
            out_key = out_key.replace('self_attn.q_proj', 'attention.wq')
        elif out_key.endswith('self_attn.v_proj'):
            out_key = out_key.replace('self_attn.v_proj', 'attention.wv')
        elif out_key.endswith('down_proj'):
            out_key = out_key.replace('down_proj', 'w2')
        elif out_key.endswith('gate_proj'):
            out_key = out_key.replace('gate_proj', 'w1')
        elif out_key.endswith('up_proj'):
            out_key = out_key.replace('up_proj', 'w3')
        elif out_key == 'embed_tokens':
            out_key = 'tok_embeddings'
        elif out_key == 'norm':
            out_key = 'norm'
        else:
            print(f"Don't know how to handle {in_key=}")
    elif out_key == 'lm_head':
        out_key = 'output'
    else:
        print(f"Don't know how to handle {in_key=}")
    return f'{out_key}.weight'


def reverse_permute(tensor: torch.Tensor, config: ModelConfig, is_kv: bool = False) -> torch.Tensor:
    """
    Reverse permute operation with adaptive dimensions based on model size and weight type.

    Args:
        tensor: Input tensor to be permuted
        config: ModelConfig object containing model parameters
        is_kv: Boolean flag indicating if the tensor is for key/value weights

    Returns:
        torch.Tensor: Permuted tensor
    """
    # Determine the number of heads and dimensions based on whether it's a KV weight
    if is_kv:
        n_heads = config.n_kv_heads
        dim1 = (config.dim // config.n_heads) * config.n_kv_heads
    else:
        n_heads = config.n_heads
        dim1 = config.dim

    dim2 = config.dim

    # Perform the reverse permutation
    return tensor.view(n_heads, 2, dim1 // n_heads // 2, dim2).transpose(1, 2).reshape(dim1, dim2)

def fixed_get_imports(filename: str | os.PathLike) -> list[str]:
    """Work around for https://huggingface.co/microsoft/phi-1_5/discussions/72."""
    if not str(filename).endswith("/modeling_deepseek.py"):
        return get_imports(filename)
    imports = get_imports(filename)
    imports.remove("flash_attn")
    return imports

def download_weights(model_size: str = "1B", out_dir: Optional[Path] = None):
    """
    Download and process weights for specified model size.

    Args:
        model_size: One of "1B" or "3B"
        out_dir: Optional output directory. If None, uses f'weights/{model_size}-Instruct'
    """
    if model_size not in MODEL_CONFIGS:
        raise ValueError(f"Invalid model size. Choose from: {list(MODEL_CONFIGS.keys())}")

    config = MODEL_CONFIGS[model_size]
    model_id = MODEL_IDS[model_size]

    if out_dir is None:
        out_dir = Path(f'weights/{model_size}-Instruct')

    device = torch.device("cpu")
    out_dir.mkdir(parents=True, exist_ok=True)

    with patch("transformers.dynamic_module_utils.get_imports", fixed_get_imports):
        hf_model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            offload_folder="/tmp/offload",
            token=TOKEN,
            device_map='cpu'
        )

        with torch.no_grad():
            state_dict = hf_model.state_dict()
            for hf_name, param in state_dict.items():
                print(f' {hf_name}: {param.shape=}')
                name = translate_key(hf_name)
                param = param.cpu()

                # Apply reverse permute for attention weights
                if name.endswith('wq.weight'):
                    param = reverse_permute(param, config, is_kv=False)
                elif name.endswith('wk.weight'):
                    param = reverse_permute(param, config, is_kv=True)

                # Convert to bfloat16 and save
                bf16_np_out = param.cpu().view(dtype=torch.uint16).numpy().view(ml_dtypes.bfloat16)
                bf16_out = jnp.asarray(bf16_np_out, dtype=jnp.bfloat16).reshape(*param.shape)
                print(f'Writing {hf_name} as {name} to {out_dir}/{name}.npy')
                jnp.save(f'{out_dir}/{name}.npy', bf16_out)

    # Cleanup
    del hf_model
    del state_dict
    jax.clear_caches()
    torch.cuda.empty_cache()

# Load Weights

In [ ]:
from typing import List, NamedTuple
import torch
import jax
import jax.numpy as jnp
import numpy as np

import ml_dtypes

from pathlib import Path

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

class LayerWeights(NamedTuple):
    wq: torch.Tensor
    wk: torch.Tensor
    wv: torch.Tensor
    wo: torch.Tensor
    w1: torch.Tensor
    w2: torch.Tensor
    w3: torch.Tensor
    ffn_norm: torch.Tensor
    attention_norm: torch.Tensor

class XfmrWeights(NamedTuple):
    tok_embeddings: torch.Tensor
    norm: torch.Tensor
    output: torch.Tensor
    layer_weights: List[LayerWeights]

def load_weights(model_size: str = "1B", ckpt_dir: Optional[Path] = None) -> Dict:
    """
    Load weights for specified model size.

    Args:
        model_size: One of "1B", or "3B"
        ckpt_dir: Optional checkpoint directory. If None, uses f'weights/{model_size}-Instruct'

    Returns:
        Dict: Model weights
    """
    if model_size not in MODEL_CONFIGS:
        raise ValueError(f"Invalid model size. Choose from: {list(MODEL_CONFIGS.keys())}")

    if ckpt_dir is None:
        ckpt_dir = Path(f'weights/{model_size}-Instruct')

    config = MODEL_CONFIGS[model_size]
    n_layers = config.n_layers
    w = {}
    layer_weights = []
    with torch.inference_mode():
      for file in ckpt_dir.glob("*.npy"):
        name = '.'.join(str(file).split('/')[-1].split('.')[:-1])
        jax_weight = jnp.load(file=file, mmap_mode='r', allow_pickle=True)
        #print(f'JAX output (first 30): {jax_weight.flatten()[:30]}')
        np_weight = np.array(jax_weight).astype(np.float32)
        weight = torch.from_numpy(np_weight).to(torch.bfloat16).to(device)
        w[name] = weight.to(device)
      for i in range(n_layers):
        layer_weights.append(LayerWeights(
          wq=w[f'layers.{i}.attention.wq.weight'],
          wk=w[f'layers.{i}.attention.wk.weight'],
          wv=w[f'layers.{i}.attention.wv.weight'],
          wo=w[f'layers.{i}.attention.wo.weight'],
          w1=w[f'layers.{i}.feed_forward.w1.weight'],
          w2=w[f'layers.{i}.feed_forward.w2.weight'],
          w3=w[f'layers.{i}.feed_forward.w3.weight'],
          ffn_norm=w[f'layers.{i}.ffn_norm.weight'],
          attention_norm=w[f'layers.{i}.attention_norm.weight'],
        ))

      xfmr_weights = XfmrWeights(
        tok_embeddings=w['tok_embeddings.weight'],
        norm=w['norm.weight'],
        output=w['output.weight'],
        layer_weights=layer_weights
      )

      return xfmr_weights

#xfmr_weights = load_weights()

# Tokenizer

Made quite a few changes to work with tokenizer json instead of tiktoken.

In [ ]:
!wget --header="Authorization: Bearer {TOKEN}" "https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/original/tokenizer.model?download=true" -O tokenizer.model

In [ ]:
import os
from logging import getLogger
from pathlib import Path
from typing import (
  AbstractSet,
  cast,
  Collection,
  Dict,
  Iterator,
  List,
  Literal,
  Optional,
  Sequence,
  Union,
)

import tiktoken

from tiktoken.load import load_tiktoken_bpe

logger = getLogger(__name__)


# The tiktoken tokenizer can handle <=400k chars without
# pyo3_runtime.PanicException.
TIKTOKEN_MAX_ENCODE_CHARS = 400_000

# https://github.com/openai/tiktoken/issues/195
# Here we iterate over subsequences and split if we exceed the limit
# of max consecutive non-whitespace or whitespace characters.
MAX_NO_WHITESPACES_CHARS = 25_000


class Tokenizer:
  """
  Tokenizing and encoding/decoding text using the Tiktoken tokenizer.
  """

  special_tokens: Dict[str, int]

  num_reserved_special_tokens = 256

  pat_str = r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"  # noqa: E501

  def __init__(self, model_path: str):
    """
    Initializes the Tokenizer with a Tiktoken model.

    Args:
        model_path (str): The path to the Tiktoken model file.
    """
    assert os.path.isfile(model_path), model_path

    mergeable_ranks = load_tiktoken_bpe(model_path)
    num_base_tokens = len(mergeable_ranks)
    special_tokens = [
      '<|begin_of_text|>',
      '<|end_of_text|>',
      '<|reserved_special_token_0|>',
      '<|reserved_special_token_1|>',
      '<|finetune_right_pad_id|>',
      '<|step_id|>',
      '<|start_header_id|>',
      '<|end_header_id|>',
      '<|eom_id|>',  # end of message
      '<|eot_id|>',  # end of turn
      '<|python_tag|>',
    ]
    reserved_tokens = [
      f'<|reserved_special_token_{2 + i}|>'
      for i in range(self.num_reserved_special_tokens - len(special_tokens))
    ]
    special_tokens = special_tokens + reserved_tokens

    self.special_tokens = {token: num_base_tokens + i for i, token in enumerate(special_tokens)}
    self.model = tiktoken.Encoding(
      name=Path(model_path).name,
      pat_str=self.pat_str,
      mergeable_ranks=mergeable_ranks,
      special_tokens=self.special_tokens,
    )

    self.n_words: int = num_base_tokens + len(special_tokens)
    # BOS / EOS token IDs
    self.bos_id: int = self.special_tokens['<|begin_of_text|>']
    self.eos_id: int = self.special_tokens['<|end_of_text|>']
    self.eot_id: int = self.special_tokens['<|eot_id|>']
    self.eom_id: int = self.special_tokens['<|eom_id|>']
    self.python_tag_id = self.special_tokens['<|python_tag|>']
    self.pad_id: int = self.special_tokens['<|finetune_right_pad_id|>']
    self.stop_tokens = [
      self.special_tokens['<|eom_id|>'],
      self.special_tokens['<|eot_id|>'],
    ]

  def encode(
    self,
    s: str,
    *,
    bos: bool,
    eos: bool,
    allowed_special: Optional[Union[Literal['all'], AbstractSet[str]]] = None,
    disallowed_special: Union[Literal['all'], Collection[str]] = (),
  ) -> List[int]:
    """
    Encodes a string into a list of token IDs.

    Args:
        s (str): The input string to be encoded.
        bos (bool): Whether to prepend the beginning-of-sequence token.
        eos (bool): Whether to append the end-of-sequence token.
        allowed_tokens ("all"|set[str]): allowed special tokens in string
        disallowed_tokens ("all"|set[str]): special tokens that raise an error when in string

    Returns:
        list[int]: A list of token IDs.

    By default, setting disallowed_special=() encodes a string by ignoring
    special tokens. Specifically:
    - Setting `disallowed_special` to () will cause all text corresponding
      to special tokens to be encoded as natural text (insteading of raising
      an error).
    - Setting `allowed_special` to "all" will treat all text corresponding
      to special tokens to be encoded as special tokens.
    """
    if allowed_special is None:
      allowed_special = set()
    assert isinstance(s, str)

    substrs = (
      substr
      for i in range(0, len(s), TIKTOKEN_MAX_ENCODE_CHARS)
      for substr in self._split_whitespaces_or_nonwhitespaces(
        s[i : i + TIKTOKEN_MAX_ENCODE_CHARS], MAX_NO_WHITESPACES_CHARS
      )
    )
    t: List[int] = []
    for substr in substrs:
      t.extend(
        self.model.encode(
          substr,
          allowed_special=allowed_special,
          disallowed_special=disallowed_special,
        )
      )
    if bos:
      t.insert(0, self.bos_id)
    if eos:
      t.append(self.eos_id)
    return t

  def decode(self, t: Sequence[int]) -> str:
    """
    Decodes a list of token IDs into a string.

    Args:
        t (List[int]): The list of token IDs to be decoded.

    Returns:
        str: The decoded string.
    """
    # Typecast is safe here. Tiktoken doesn't do anything list-related with the sequence.
    return self.model.decode(cast(List[int], t))

  @staticmethod
  def _split_whitespaces_or_nonwhitespaces(s: str, max_consecutive_slice_len: int) -> Iterator[str]:
    """
    Splits the string `s` so that each substring contains no more than `max_consecutive_slice_len`
    consecutive whitespaces or consecutive non-whitespaces.
    """
    current_slice_len = 0
    current_slice_is_space = s[0].isspace() if len(s) > 0 else False
    slice_start = 0

    for i in range(len(s)):
      is_now_space = s[i].isspace()

      if current_slice_is_space ^ is_now_space:
        current_slice_len = 1
        current_slice_is_space = is_now_space
      else:
        current_slice_len += 1
        if current_slice_len > max_consecutive_slice_len:
          yield s[slice_start:i]
          slice_start = i
          current_slice_len = 1
    yield s[slice_start:]

# KVCache

In [ ]:
import torch
import torch.nn as nn

# Device selection, tree is like first apple silicion, then cuda, fallback is cpu.
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

class KVCache(nn.Module):
    def __init__(self, layers: int, bsz: int, max_seq_len: int, kv_heads: int, head_dim: int):
        super(KVCache, self).__init__()
        # Initialize k and v as buffers to ensure they're part of the module state
        self.register_buffer(
            'k',
            torch.zeros(
                (layers, bsz, max_seq_len, kv_heads, head_dim),
                dtype=torch.bfloat16,
                device=device
            )
        )
        self.register_buffer(
            'v',
            torch.zeros(
                (layers, bsz, max_seq_len, kv_heads, head_dim),
                dtype=torch.bfloat16,
                device=device
            )
        )

    @classmethod
    def new(cls, layers: int, bsz: int, max_seq_len: int, kv_heads: int, head_dim: int) -> 'KVCache':
        """Creates a new KVCache instance with initialized k and v tensors."""
        return cls(layers, bsz, max_seq_len, kv_heads, head_dim)

    def update(
        self,
        xk: torch.Tensor,
        xv: torch.Tensor,
        layer_idx: int,
        cur_pos: int,
        n_rep: int
    ):
        """
        Updates the cache with new key and value tensors.

        Args:
            xk (torch.Tensor): New key tensor to insert. Shape should align with (bsz, insert_len, kv_heads, head_dim).
            xv (torch.Tensor): New value tensor to insert. Shape should align with (bsz, insert_len, kv_heads, head_dim).
            layer_idx (int): The index of the layer to update.
            cur_pos (int): The current position in the sequence to start inserting.
            n_rep (int): The number of times to repeat the keys and values along the sequence dimension.

        Returns:
            Tuple[torch.Tensor, torch.Tensor]:
                - keys: Updated or repeated keys tensor.
                - values: Updated or repeated values tensor.
        """
        # Ensure xk and xv have the correct device and dtype
        xk = xk.to(self.k.dtype)
        xv = xv.to(self.v.dtype)

        # Update the k and v tensors in the specified layer and position
        insert_len = xk.size(1)  # Assuming xk shape is (bsz, insert_len, kv_heads, head_dim)
        self.k[layer_idx, :, cur_pos:cur_pos+insert_len, :, :] = xk
        self.v[layer_idx, :, cur_pos:cur_pos+insert_len, :, :] = xv

        if cur_pos == 0:
            # If inserting at the beginning, repeat the new keys and values
            keys = xk.repeat_interleave(n_rep, dim=2)
            values = xv.repeat_interleave(n_rep, dim=2)
        else:
            # Otherwise, repeat the existing keys and values from the cache
            keys = self.k[layer_idx].repeat_interleave(n_rep, dim=2)
            values = self.v[layer_idx].repeat_interleave(n_rep, dim=2)

        return keys, values, self

    def clear(self):
        """Resets the k and v caches to zeros."""
        self.k.zero_()
        self.v.zero_()


# Attention Stats

In [ ]:
import torch

# Device selection, tree is like first apple silicion, then cuda, fallback is cpu.
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

from typing import NamedTuple

class AttnStats(NamedTuple):
    entropy: torch.Tensor  # (bsz, n_layers, num_heads)
    varentropy: torch.Tensor  # (bsz, n_layers, num_heads)
    n_layers: int
    n_heads: int

    @classmethod
    def new(cls, bsz: int, n_layers: int, n_heads: int) -> 'AttnStats':
        return cls(
            entropy=torch.zeros((bsz, n_layers, n_heads), dtype=torch.float32, device=device),
            varentropy=torch.zeros((bsz, n_layers, n_heads), dtype=torch.float32, device=device),
            n_layers=n_layers,
            n_heads=n_heads
        )

    @property
    def avg_entropy(self):
        return self.entropy.sum(dim=-1, keepdim=False)  # Average across heads

    @property
    def std_error(self):
        return torch.sqrt(torch.mean(self.varentropy)) / (self.n_heads * self.n_layers)

    def update(self, scores: torch.Tensor, layer_idx: int):
        # scores shape: (bsz, n_heads, seqlen, n_words)
        probs = torch.nn.functional.softmax(scores, dim=-1)
        new_entropy = -torch.sum(torch.where(probs > 0, probs * torch.log(probs), torch.tensor(0.0)), dim=-1)
        new_varentropy = torch.sum(probs * (torch.log(probs) + new_entropy.unsqueeze(-1))**2, dim=-1)

        # Update entropy and varentropy tensors
        self.entropy[:, layer_idx, :] = new_entropy
        self.varentropy[:, layer_idx, :] = new_varentropy

        return self

# Model

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

DEFAULT_MASK_VALUE = -0.7 * float(torch.finfo(torch.float32).max)

# Device selection, tree is like first apple silicion, then cuda, fallback is cpu.
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

from typing import Tuple, Optional

def rms_norm(x: torch.Tensor, w: torch.Tensor, eps: float = 1e-5) -> torch.Tensor:
  return w * (x * torch.rsqrt(torch.pow(x, 2).mean(-1, keepdim=True) + eps))

def apply_rotary_emb(xq: torch.Tensor, xk: torch.Tensor, freqs_cis: torch.Tensor, dtype: torch.dtype = torch.float32) -> Tuple[torch.Tensor, torch.Tensor]:
    reshape_xq = xq.float().reshape(*xq.shape[:-1], -1, 2)
    reshape_xk = xk.float().reshape(*xk.shape[:-1], -1, 2)
    xq_ = torch.complex(reshape_xq[..., 0], reshape_xq[..., 1])
    xk_ = torch.complex(reshape_xk[..., 0], reshape_xk[..., 1])
    xq_out = xq_ * freqs_cis.unsqueeze(0).unsqueeze(2)
    xk_out = xk_ * freqs_cis.unsqueeze(0).unsqueeze(2)
    xq_out = torch.stack((xq_out.real, xq_out.imag), dim=-1).reshape(*xq_out.shape[:-1], -1)
    xk_out = torch.stack((xk_out.real, xk_out.imag), dim=-1).reshape(*xk_out.shape[:-1], -1)
    return xq_out.to(dtype), xk_out.to(dtype)

def attention(x: torch.Tensor, layer_weights: LayerWeights, model_params, cur_pos: int, layer_idx: int, freqs_cis: torch.Tensor, kvcache: KVCache, attn_mask: Optional[torch.Tensor] = None) -> Tuple[torch.Tensor, KVCache, torch.Tensor]:
    # Check if x is 2D or 3D and adjust accordingly
    if x.dim() == 2:
        bsz = 1
        seq_len, dim = x.shape
        x = x.unsqueeze(0)  # Add batch dimension
    else:
        bsz, seq_len, dim = x.shape

    n_rep = model_params.n_local_heads // model_params.n_local_kv_heads
    xq = F.linear(x, layer_weights.wq).view(bsz, seq_len, model_params.n_local_heads, model_params.head_dim)
    xk = F.linear(x, layer_weights.wk).view(bsz, seq_len, model_params.n_local_kv_heads, model_params.head_dim)
    xv = F.linear(x, layer_weights.wv).view(bsz, seq_len, model_params.n_local_kv_heads, model_params.head_dim)
    xq, xk = apply_rotary_emb(xq, xk, freqs_cis=freqs_cis, dtype=xq.dtype)
    keys, values, kvcache = kvcache.update(xk, xv, layer_idx, cur_pos, n_rep)
    xq = xq.permute(0, 2, 1, 3)  # (bs, n_heads, seqlen, head_dim)
    keys = keys.permute(0, 2, 3, 1)  # (bs, n_heads, head_dim, cache_len + seqlen)
    values = values.permute(0, 2, 1, 3)  # (bs, n_heads, cache_len + seqlen, head_dim)
    scores = torch.matmul(xq, keys)
    pre_scores = scores / math.sqrt(model_params.head_dim)
    scores = pre_scores.to(torch.float32)  # Always do attention softmax at float32
    if cur_pos == 0:
        scores = scores + attn_mask
    mask = torch.where(scores != 0.0, scores, DEFAULT_MASK_VALUE)
    padded_logits = torch.where((mask >= DEFAULT_MASK_VALUE * 0.5), scores, DEFAULT_MASK_VALUE)
    scores = F.softmax(padded_logits, dim=-1).to(x.dtype)
    output = torch.matmul(scores.to(values.dtype), values)
    output = output.transpose(1, 2).contiguous().view(bsz, seq_len, -1)
    out = F.linear(output, layer_weights.wo)

    # If input was 2D, remove the batch dimension from the output
    if x.dim() == 2:
        out = out.squeeze(0)

    return out, kvcache, pre_scores

def feed_forward(x: torch.Tensor, layer_weights: LayerWeights) -> torch.Tensor:
 return F.linear(F.silu(F.linear(x, layer_weights.w1)) * F.linear(x, layer_weights.w3), layer_weights.w2)

def xfmr(xfmr_weights: XfmrWeights, model_params: ModelParams, tokens: torch.Tensor, cur_pos: int, freqs_cis: torch.Tensor, kvcache: KVCache, attn_mask: Optional[torch.Tensor]=None) -> Tuple[torch.Tensor, KVCache, torch.Tensor, AttnStats]:
    h = xfmr_weights.tok_embeddings[tokens]
    attn_stats = AttnStats.new(
        bsz=tokens.shape[0],
        n_layers=model_params.n_layers,
        n_heads=model_params.n_local_heads
    )
    for i in range(model_params.n_layers):
        norm_x = rms_norm(h, xfmr_weights.layer_weights[i].attention_norm)
        h_attn, kvcache, scores = attention(norm_x, xfmr_weights.layer_weights[i], model_params, cur_pos, i, freqs_cis, kvcache, attn_mask=attn_mask)
        attn_stats = attn_stats.update(scores[:,:,-1,:], i)
        h = h + h_attn
        h = h + feed_forward(rms_norm(h, xfmr_weights.layer_weights[i].ffn_norm), xfmr_weights.layer_weights[i])
    logits = F.linear(rms_norm(h, xfmr_weights.norm), xfmr_weights.output)
    return logits, kvcache, scores, attn_stats

#Sampler Config

In [ ]:
class SamplerConfig:
    def __init__(self, model_size: str = "1B"):
        """
        Initialize SamplerConfig with specified model size.

        Args:
            model_size: One of "1B", or "3B"
        """
        self.model_size = model_size  # Store model_size as instance variable

        if self.model_size == "1B":
            """
            Configuration for the sampling strategy, including threshold values for various metrics
            and adaptive sampling parameters.
            """
            self.temperature = 0.666
            self.top_p = 0.90
            self.top_k = 27
            self.min_p = 0.03

            self.low_logits_entropy_threshold = 0.01
            self.medium_logits_entropy_threshold = 0.7
            self.high_logits_entropy_threshold = 2.1

            self.low_logits_varentropy_threshold = 0.05
            self.medium_logits_varentropy_threshold = 2.0
            self.high_logits_varentropy_threshold = 5.8

            self.low_attention_entropy_threshold = 11.915
            self.medium_attention_entropy_threshold = 11.921
            self.high_attention_entropy_threshold = 11.926

            self.low_attention_varentropy_threshold = 0.001
            self.medium_attention_varentropy_threshold = 0.0045
            self.high_attention_varentropy_threshold = 0.009

            self.low_agreement_threshold = 1.8e-06
            self.medium_agreement_threshold = 3.8e-06
            self.high_agreement_threshold = 4.8e-06

            self.low_interaction_strength_threshold = 0.18
            self.medium_interaction_strength_threshold = 0.227
            self.high_interaction_strength_threshold = 0.244

            self.high_entropy_attention_offset = 1.3
            self.high_entropy_attention_coefficient = 0.2

            self.low_entropy_interaction_strength_offset = 1.2
            self.low_entropy_interaction_strength_coefficient = 0.3

            self.high_entropy_varentropy_attention_offset = 2.0
            self.high_entropy_varentropy_attention_coefficient = 0.5

            self.n_adaptive_samples = 5

            self.adaptive_temperature_logits_coefficient = 0.3
            self.adaptive_temperature_attention_coefficient = 0.2
            self.adaptive_temperature_agreement_coefficient = 0.2
            self.adaptive_top_p_coefficient = 0.1
            self.adaptive_top_k_interaction_coefficient = 0.3
            self.adaptive_top_k_agreement_coefficient = 0.2
            self.adaptive_min_p_coefficient = 0.5
            self.adaptive_score_logits_entropy_coefficient = 0.1
            self.adaptive_score_attention_entropy_coefficient = 0.2
            self.adaptive_score_logits_varentropy_coefficient = 0.3
            self.adaptive_score_attention_varentropy_coefficient = 0.4
            self.adaptive_score_agreement_coefficient = 0.5
            self.adaptive_score_interaction_strength_coefficient = 0.6


        elif self.model_size == "3B":
            self.temperature = 0.666
            self.top_p = 0.90
            self.top_k = 27
            self.min_p = 0.03

            self.low_logits_entropy_threshold = 0.01
            self.medium_logits_entropy_threshold = 0.7
            self.high_logits_entropy_threshold = 2.1

            self.low_logits_varentropy_threshold = 0.05
            self.medium_logits_varentropy_threshold = 2.0
            self.high_logits_varentropy_threshold = 5.8

            self.low_attention_entropy_threshold = 11.915
            self.medium_attention_entropy_threshold = 11.921
            self.high_attention_entropy_threshold = 11.926

            self.low_attention_varentropy_threshold = 0.001
            self.medium_attention_varentropy_threshold = 0.0045
            self.high_attention_varentropy_threshold = 0.009

            self.low_agreement_threshold = 1.8e-06
            self.medium_agreement_threshold = 3.8e-06
            self.high_agreement_threshold = 4.8e-06

            self.low_interaction_strength_threshold = 0.18
            self.medium_interaction_strength_threshold = 0.227
            self.high_interaction_strength_threshold = 0.244

            self.high_entropy_attention_offset = 1.3
            self.high_entropy_attention_coefficient = 0.2

            self.low_entropy_interaction_strength_offset = 1.2
            self.low_entropy_interaction_strength_coefficient = 0.3

            self.high_entropy_varentropy_attention_offset = 2.0
            self.high_entropy_varentropy_attention_coefficient = 0.5

            self.n_adaptive_samples = 5

            self.adaptive_temperature_logits_coefficient = 0.3
            self.adaptive_temperature_attention_coefficient = 0.2
            self.adaptive_temperature_agreement_coefficient = 0.2
            self.adaptive_top_p_coefficient = 0.1
            self.adaptive_top_k_interaction_coefficient = 0.3
            self.adaptive_top_k_agreement_coefficient = 0.2
            self.adaptive_min_p_coefficient = 0.5
            self.adaptive_score_logits_entropy_coefficient = 0.1
            self.adaptive_score_attention_entropy_coefficient = 0.2
            self.adaptive_score_logits_varentropy_coefficient = 0.3
            self.adaptive_score_attention_varentropy_coefficient = 0.4
            self.adaptive_score_agreement_coefficient = 0.5
            self.adaptive_score_interaction_strength_coefficient = 0.6

        else:
            raise ValueError(f"Invalid model size: {model_size}. Choose from: 1B, 3B")

# Sampler

In [ ]:
import torch
import torch.nn.functional as F
from typing import Tuple, Dict
from enum import Enum

# Device selection
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

LN_2 = 0.69314718056  # ln(2) = 1.0 / LOG2_E

class SamplerState(Enum):
    FLOWING = "Flowing with unspoken intent"
    TREADING = "Treading carefully, asking clarifying questions"
    EXPLORING = "Exploring forks in the path"
    RESAMPLING = "Resampling in the mist"
    ADAPTIVE = "Adaptive Sampling"

def calculate_varentropy_logsoftmax(logits: torch.Tensor, axis: int = -1) -> Tuple[torch.Tensor, torch.Tensor]:
    """Calculate the entropy and varentropy of the probability distribution using logsoftmax."""
    log_probs = F.log_softmax(logits, dim=axis)
    probs = torch.exp(log_probs)
    entropy = -torch.sum(probs * log_probs, dim=axis) / LN_2  # Convert to base-2
    varentropy = torch.sum(probs * (log_probs / LN_2 + entropy.unsqueeze(-1))**2, dim=axis)
    return entropy, varentropy

def multinomial_sample_one(probs_sort: torch.Tensor, generator: torch.Generator) -> torch.Tensor:
    """Samples one token from a multinomial distribution with sorted probabilities."""
    q = torch.rand(probs_sort.shape, generator=generator, device=probs_sort.device)
    return torch.argmax(probs_sort / q, dim=-1, keepdim=True).to(torch.int32)

def _sample(logits: torch.Tensor, temperature: float, top_p: float, top_k: int, min_p: float, generator: torch.Generator = None) -> torch.Tensor:
    bsz = logits.shape[0]
    logit = logits[:, -1]
    probs = F.softmax(logit / temperature, dim=-1)

    # Apply min_p samplinga
    if min_p > 0.0:
        p_max = torch.max(probs, dim=-1, keepdim=True).values
        indices_to_remove = probs < (min_p * p_max)
        logit = torch.where(indices_to_remove, torch.full_like(logit, float('-inf')), logit)
        probs = F.softmax(logit, dim=-1)

    # Apply top-k sampling
    top_k_probs, top_k_indices = torch.topk(probs, k=min(top_k, probs.shape[-1]))
    probs_sort = torch.flip(top_k_probs, dims=[-1])
    probs_idx = torch.flip(top_k_indices, dims=[-1])
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    # Apply top-p sampling
    mask = torch.where(probs_sum - probs_sort > top_p, torch.tensor(1.0, device=device), torch.tensor(0.0, device=device))
    probs_sort = probs_sort * (1 - mask)
    probs_sort = probs_sort / torch.sum(probs_sort, dim=-1, keepdim=True)
    next_token = multinomial_sample_one(probs_sort, generator)
    next_token_g = torch.gather(probs_idx, -1, next_token.reshape(bsz, 1).to(torch.int64))
    return next_token_g.to(torch.int32)

def calculate_metrics(logits: torch.Tensor, attention_scores: torch.Tensor) -> Dict[str, torch.Tensor]:
    entropy, varentropy = calculate_varentropy_logsoftmax(logits)
    attention_probs = F.softmax(attention_scores, dim=-1)
    attn_entropy = -torch.sum(attention_probs * torch.log2(torch.clamp(attention_probs, 1e-10, 1.0)), dim=-1)
    attn_varentropy = torch.var(attn_entropy, dim=1)

    attn_varentropy = torch.where(torch.isnan(attn_varentropy), torch.zeros_like(attn_varentropy), attn_varentropy)
    mean_attention = torch.mean(attention_probs, dim=1)
    agreement = torch.mean(torch.abs(attention_probs - mean_attention.unsqueeze(1)), dim=(1, 2))

    interaction_strength = torch.mean(torch.abs(attention_scores), dim=(1, 2, 3))

    return {
        "logits_entropy": torch.mean(entropy),
        "logits_varentropy": torch.mean(varentropy),
        "attn_entropy": torch.mean(attn_entropy),
        "attn_varentropy": torch.mean(attn_varentropy),
        "agreement": torch.mean(agreement),
        "interaction_strength": interaction_strength
    }

def adaptive_sample(logits: torch.Tensor, temperature: float, epsilon: float = 0.01,
                   generator: torch.Generator = None) -> torch.Tensor:
    """
    Perform adaptive sampling by dynamically adjusting the candidate set size based on entropy and varentropy.
    """
    bsz = logits.shape[0]
    logit = logits[:, -1]
    probs = F.softmax(logit / temperature, dim=-1)

    # Sort tokens by probability
    sorted_probs, sorted_indices = torch.topk(probs, k=probs.shape[-1], dim=-1)

    # Initialize candidate set size
    candidate_mask = torch.zeros_like(sorted_probs, dtype=torch.bool, device=logits.device)
    cumulative_entropy = torch.zeros(bsz, device=logits.device)
    cumulative_varentropy = torch.zeros(bsz, device=logits.device)
    # Initial entropy calculation
    previous_entropy = -torch.sum(sorted_probs[0] * torch.log2(torch.clamp(sorted_probs[0], 1e-10, 1.0)))

    i = 0
    while i < sorted_probs.shape[-1]:
        current_prob = sorted_probs[:, i]

        # Update entropy and varentropy with current token
        current_entropy = -torch.sum(current_prob * torch.log2(torch.clamp(current_prob, 1e-10, 1.0)))
        current_varentropy = torch.sum(current_prob * (torch.log2(torch.clamp(current_prob, 1e-10, 1.0)) +
                                                     cumulative_entropy.unsqueeze(-1))**2)

        entropy_reduction = cumulative_entropy - current_entropy
        varentropy_reduction = cumulative_varentropy - current_varentropy

        # Update mask where entropy reduction is sufficient
        candidate_mask[:, i] = entropy_reduction >= epsilon

        # Update cumulative values
        cumulative_entropy = torch.where(entropy_reduction >= epsilon,
                                       cumulative_entropy.clone(),
                                       current_entropy)
        cumulative_varentropy = torch.where(entropy_reduction >= epsilon,
                                          cumulative_varentropy.clone(),
                                          current_varentropy)

        # Check continuation condition
        if not torch.any(entropy_reduction >= epsilon) or i >= sorted_probs.shape[-1] - 1:
            break

        i += 1

    # Mask out tokens not in the candidate set
    candidate_probs = sorted_probs * candidate_mask.float()
    candidate_probs = candidate_probs / torch.sum(candidate_probs, dim=-1, keepdim=True)

    # Sample from the final candidate set
    next_token = multinomial_sample_one(candidate_probs, generator)
    next_token_g = torch.gather(sorted_indices, -1, next_token.to(torch.int64))

    return next_token_g.to(torch.int32)

def sample(gen_tokens: torch.Tensor, logits: torch.Tensor, attention_scores: torch.Tensor, cfg: SamplerConfig, entropix_cfg: EntropixConfig,
           clarifying_question_token: int = 2026, generator: torch.Generator = torch.Generator(device=device).manual_seed(1337)) -> Tuple[torch.Tensor, SamplerState]:
    metrics = calculate_metrics(logits, attention_scores)
    ent, vent = metrics["logits_entropy"], metrics["logits_varentropy"]
    attn_ent, attn_vent = metrics["attn_entropy"], metrics["attn_varentropy"]
    agreement = metrics["agreement"]
    interaction_strength = metrics["interaction_strength"]

    # Low Entropy, Low Varentropy: "flowing with unspoken intent"
    if entropix_cfg.state_flowing and (ent < cfg.low_logits_entropy_threshold and
        vent < cfg.low_logits_varentropy_threshold and
        attn_ent < cfg.low_attention_entropy_threshold and
        attn_vent < cfg.low_attention_varentropy_threshold and
        (not entropix_cfg.state_extras_agreement or agreement < cfg.low_agreement_threshold) and
        (not entropix_cfg.state_extras_interaction_strength or interaction_strength < cfg.low_interaction_strength_threshold)):
        sampler_state = SamplerState.FLOWING
        sampled_token = torch.argmax(logits[:, -1], dim=-1, keepdim=True).to(torch.int32)
        return sampled_token, sampler_state

    # High Entropy, Low Varentropy: "treading carefully, asking clarifying questions"
    elif entropix_cfg.state_treading and (ent > cfg.high_logits_entropy_threshold and
          vent < cfg.low_logits_varentropy_threshold and
          attn_ent < cfg.low_attention_entropy_threshold and
          attn_vent < cfg.low_attention_varentropy_threshold and
          (not entropix_cfg.state_extras_agreement or agreement < cfg.low_agreement_threshold) and
          (not entropix_cfg.state_extras_interaction_strength or interaction_strength < cfg.low_interaction_strength_threshold)):
        sampler_state = SamplerState.TREADING
        # Insert a clarifying question token if not already present
        if not torch.isin(gen_tokens[:, -1], torch.tensor([clarifying_question_token], device=device)).any():
            sampled_token = torch.tensor([[clarifying_question_token]], dtype=torch.int32, device=device)
            return sampled_token, sampler_state
        else:
            # If we've just asked a question, sample with slightly higher temperature
            temp_adj = cfg.high_entropy_attention_offset + cfg.high_entropy_attention_coefficient * attn_ent
            sampled_token = _sample(
                logits,
                temperature=min(1.5, cfg.temperature * temp_adj),
                top_p=cfg.top_p,
                top_k=cfg.top_k,
                min_p=cfg.min_p,
                generator=generator
            )
            return sampled_token, sampler_state

    # Low Entropy, High Varentropy: "exploring forks in the path"
    elif entropix_cfg.state_exploring and (ent < cfg.high_logits_entropy_threshold and
          vent > cfg.high_logits_varentropy_threshold and
          attn_ent < cfg.low_attention_entropy_threshold and
          attn_vent > cfg.high_attention_varentropy_threshold and
          (not entropix_cfg.state_extras_agreement or agreement < cfg.low_agreement_threshold) and
          (not entropix_cfg.state_extras_interaction_strength or interaction_strength > cfg.low_interaction_strength_threshold)):
        sampler_state = SamplerState.EXPLORING
        temp_adj = cfg.low_entropy_interaction_strength_offset + cfg.low_entropy_interaction_strength_coefficient * interaction_strength
        top_k_adj = max(5, int(cfg.top_k * (1 + 0.5 * (1 - agreement))))
        sampled_token = _sample(
            logits,
            temperature=min(1.5, cfg.temperature * temp_adj),
            top_p=cfg.top_p,
            top_k=top_k_adj,
            min_p=cfg.min_p,
            generator=generator
        )
        return sampled_token, sampler_state

    # High Entropy, High Varentropy: "resampling in the mist"
    elif entropix_cfg.state_resampling and (ent > cfg.medium_logits_entropy_threshold and
          vent > cfg.high_logits_varentropy_threshold and
          attn_ent > cfg.high_attention_entropy_threshold and
          attn_vent > cfg.high_attention_varentropy_threshold and
          (not entropix_cfg.state_extras_agreement or agreement > cfg.high_agreement_threshold) and
          (not entropix_cfg.state_extras_interaction_strength or interaction_strength > cfg.high_interaction_strength_threshold)):
        sampler_state = SamplerState.RESAMPLING
        # Use high temperature and adjusted top_p based on attention metrics
        temp_adj = cfg.high_entropy_varentropy_attention_offset + cfg.high_entropy_varentropy_attention_coefficient * attn_vent
        top_p_adj = max(0.5, cfg.top_p - cfg.high_entropy_attention_coefficient * attn_ent)
        sampled_token = _sample(
            logits,
            temperature=max(2.0, cfg.temperature * temp_adj),
            top_p=top_p_adj,
            top_k=cfg.top_k,
            min_p=cfg.min_p,
            generator=generator
        )
        return sampled_token, sampler_state

    # All other cases: use adaptive sampling
    else:
        sampler_state = SamplerState.ADAPTIVE
        '''temperature = 0.666
        sampled_token = adaptive_sample(
            logits,
            temperature=temperature,
            epsilon=0.1,
            generator=generator
        )'''
        logits_uncertainty = ent + vent
        attn_uncertainty = attn_ent + attn_vent

        temperature = cfg.temperature * (
            1 +
            cfg.adaptive_temperature_logits_coefficient * ent +
            cfg.adaptive_temperature_attention_coefficient * attn_ent -
            cfg.adaptive_temperature_agreement_coefficient * agreement
        )
        top_p = torch.clamp(
            (cfg.top_p * (1 + cfg.adaptive_top_p_coefficient * attn_vent)).clone().detach(),
            0.1,
            1.0
        )
        top_k = int(torch.clamp(
            torch.round(torch.tensor(cfg.top_k) * (
                1 +
                cfg.adaptive_top_k_interaction_coefficient * interaction_strength.item() -
                cfg.adaptive_top_k_agreement_coefficient * agreement.item()
            )),
            min=1,
            max=100
        ).item())
        min_p = torch.clamp(
            (cfg.min_p * (1 - cfg.adaptive_min_p_coefficient * vent)).clone().detach(),
            0.01,
            0.5
        )

        samples = []
        for _ in range(cfg.n_adaptive_samples):
            sample = _sample(
                logits,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                min_p=min_p,
                generator=generator
            )
            samples.append(sample)

        def score_sample(sample):
            # Ensure sample is a 1D tensor of indices
            sample_indices = sample.view(-1).to(torch.long)

            # Create one-hot encoding
            one_hot = F.one_hot(sample_indices, num_classes=logits.shape[-1])

            # Calculate log probability
            log_probs = F.log_softmax(logits[:, -1], dim=-1)
            log_prob = torch.sum(log_probs * one_hot, dim=-1)

            confidence_score = (
                (1 - ent / cfg.high_logits_entropy_threshold) * cfg.adaptive_score_logits_entropy_coefficient +
                (1 - attn_ent / cfg.high_attention_entropy_threshold) * cfg.adaptive_score_attention_entropy_coefficient +
                (1 - vent / cfg.high_logits_varentropy_threshold) * cfg.adaptive_score_logits_varentropy_coefficient +
                (1 - attn_vent / cfg.high_attention_varentropy_threshold) * cfg.adaptive_score_attention_varentropy_coefficient +
                (agreement / cfg.high_agreement_threshold) * cfg.adaptive_score_agreement_coefficient +
                (interaction_strength / cfg.high_interaction_strength_threshold) * cfg.adaptive_score_interaction_strength_coefficient
            )
            return log_prob + confidence_score

        sample_scores = torch.stack([score_sample(sample) for sample in samples])
        best_sample_idx = torch.argmax(sample_scores)
        sampled_token = samples[best_sample_idx]
        return sampled_token, sampler_state

# Main

In [ ]:
from typing import NamedTuple, Optional, Tuple
import os

import plotly.graph_objects as go
import numpy as np
import json
from datetime import datetime

import torch
import torch.nn.functional as F

import math

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.patches import Patch

from pathlib import Path
from functools import partial

from IPython.display import display, Markdown

# Device selection, tree is like first apple silicion, then cuda, fallback is cpu.
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using device: {device}")
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()
torch.set_float32_matmul_precision('high')

def apply_scaling(freqs: torch.Tensor) -> torch.Tensor:
    SCALE_FACTOR = 8.0
    LOW_FREQ_FACTOR = 1.0
    HIGH_FREQ_FACTOR = 4.0
    OLD_CONTEXT_LEN = 2048  # original llama3 length

    low_freq_wavelen = OLD_CONTEXT_LEN / LOW_FREQ_FACTOR
    high_freq_wavelen = OLD_CONTEXT_LEN / HIGH_FREQ_FACTOR

    def scale_freq(freq: torch.Tensor) -> torch.Tensor:
        wavelen = 2 * torch.pi / freq

        # Calculate smooth factor
        smooth = (OLD_CONTEXT_LEN / wavelen - LOW_FREQ_FACTOR) / (HIGH_FREQ_FACTOR - LOW_FREQ_FACTOR)
        smooth = torch.clamp(smooth, 0.0, 1.0)  # Ensure smooth is between 0 and 1

        # Calculate scaled frequency
        scaled = (1 - smooth) * freq / SCALE_FACTOR + smooth * freq

        # Apply conditional scaling
        scaled = torch.where(
            wavelen < high_freq_wavelen,
            freq,  # No scaling
            torch.where(
                wavelen > low_freq_wavelen,
                freq / SCALE_FACTOR,  # Apply scaling factor
                scaled  # Apply smooth scaling
            )
        )
        return scaled

    scaled_freqs = torch.vmap(scale_freq)(freqs)

    return scaled_freqs

def precompute_freqs_cis(dim: int, end: int, theta: float = 10000.0, use_scaled: bool = False, dtype: torch.dtype = torch.float32) -> torch.Tensor:
    freqs = 1.0 / (theta ** (torch.arange(0, dim, 2, dtype=dtype, device=device)[: (dim // 2)] / dim))
    if use_scaled:
        freqs = apply_scaling(freqs)

    t = torch.arange(end, dtype=dtype, device=device).unsqueeze(1)  # Shape: (end, 1)
    freqs = freqs.unsqueeze(0)  # Shape: (1, dim//2)
    freqs = t * freqs  # Broadcasting to shape: (end, dim//2)
    return torch.exp(1j * freqs)

def build_attn_mask(seqlen: int, start_pos: int) -> torch.Tensor:
  mask = None
  if seqlen > 1:
      mask = torch.full((seqlen, seqlen), float("-inf"))
      mask = torch.triu(mask, diagonal=1)
      mask = torch.hstack([torch.zeros((seqlen, start_pos)), mask]).to(torch.float32).to(device)
  return mask

class EntropixModel:
    def __init__(self, model_size: str = "1B"):
        """
        Initialize EntropixModel with specified model size.

        Args:
            model_size: One of "1B", or "3B"
        """
        if model_size not in MODEL_CONFIGS:
            raise ValueError(f"Invalid model size. Choose from: {list(MODEL_CONFIGS.keys())}")

        self.model_size = model_size
        self.config = MODEL_CONFIGS[model_size]
        self.model_params = get_model_params(self.config)
        self.xfmr_weights = load_weights(model_size=model_size)
        self.tokenizer = Tokenizer('tokenizer.model')
        self.sampler_config = SamplerConfig(model_size)
        self.entropix_config = EntropixConfig()
        self.generator = torch.Generator(device=device).manual_seed(1337)

    def visualize_token_entropy_varentropy(self, metrics_data, generated_tokens):
        # Extract data
        entropies = np.array(metrics_data['logits_entropy'])
        varentropies = np.array(metrics_data['logits_varentropy'])
        attention_entropies = np.array(metrics_data['attention_entropy'])
        attention_varentropies = np.array(metrics_data['attention_varentropy'])

        # Ensure all arrays have the same length
        min_length = min(len(entropies), len(varentropies), len(attention_entropies), len(attention_varentropies), len(generated_tokens))
        entropies = entropies[:min_length]
        varentropies = varentropies[:min_length]
        attention_entropies = attention_entropies[:min_length]
        attention_varentropies = attention_varentropies[:min_length]
        generated_tokens = generated_tokens[:min_length]

        positions = np.arange(min_length)

        # Create hover text
        hover_text = [
            f"Token: {self.tokenizer.decode([token]) or 'Unknown'}<br>"
            f"Position: {i}<br>"
            f"Logits Entropy: {entropies[i]:.4f}<br>"
            f"Logits Varentropy: {varentropies[i]:.4f}<br>"
            f"Attention Entropy: {attention_entropies[i]:.4f}<br>"
            f"Attention Varentropy: {attention_varentropies[i]:.4f}"
            for i, token in enumerate(generated_tokens)
        ]

        # Create the 3D scatter plot
        fig = go.Figure()

        # Add logits entropy/varentropy scatter
        fig.add_trace(go.Scatter3d(
            x=entropies,
            y=varentropies,
            z=positions,
            mode='markers',
            marker=dict(
                size=5,
                color=entropies,
                colorscale='Viridis',
                opacity=0.8,
                colorbar=dict(title="Logits Entropy", x=0.85),
            ),
            text=hover_text,
            hoverinfo='text',
            name='Logits Entropy/Varentropy'
        ))

        # Add attention entropy/varentropy scatter
        fig.add_trace(go.Scatter3d(
            x=attention_entropies,
            y=attention_varentropies,
            z=positions,
            mode='markers',
            marker=dict(
                size=5,
                color=attention_entropies,
                colorscale='Plasma',
                opacity=0.8,
                colorbar=dict(title="Attention Entropy", x=1.0),
            ),
            text=hover_text,
            hoverinfo='text',
            name='Attention Entropy/Varentropy'
        ))

        # Calculate the limits for x, y, and z
        logits_x_min, logits_x_max = min(entropies), max(entropies)
        logits_y_min, logits_y_max = min(varentropies), max(varentropies)
        attention_x_min, attention_x_max = min(attention_entropies), max(attention_entropies)
        attention_y_min, attention_y_max = min(attention_varentropies), max(attention_varentropies)
        z_min, z_max = min(positions), max(positions)

        # Function to create threshold planes
        def create_threshold_plane(threshold, axis, color, name, data_type):
            if data_type == 'logits':
                x_min, x_max = logits_x_min, logits_x_max
                y_min, y_max = logits_y_min, logits_y_max
            else:  # attention
                x_min, x_max = attention_x_min, attention_x_max
                y_min, y_max = attention_y_min, attention_y_max

            if axis == 'x':
                return go.Surface(
                    x=[[threshold, threshold], [threshold, threshold]],
                    y=[[y_min, y_max], [y_min, y_max]],
                    z=[[z_min, z_min], [z_max, z_max]],
                    colorscale=[[0, color], [1, color]],
                    showscale=False,
                    name=name,
                    visible=False
                )
            elif axis == 'y':
                return go.Surface(
                    x=[[x_min, x_max], [x_min, x_max]],
                    y=[[threshold, threshold], [threshold, threshold]],
                    z=[[z_min, z_min], [z_max, z_max]],
                    colorscale=[[0, color], [1, color]],
                    showscale=False,
                    name=name,
                    visible=False
                )

        # Add threshold planes
        thresholds = [
            ('logits_entropy', 'x', [
                (self.sampler_config.low_logits_entropy_threshold, 'rgba(255, 0, 0, 0.2)'),
                (self.sampler_config.medium_logits_entropy_threshold, 'rgba(0, 255, 0, 0.2)'),
                (self.sampler_config.high_logits_entropy_threshold, 'rgba(0, 0, 255, 0.2)')
            ], 'logits'),
            ('logits_varentropy', 'y', [
                (self.sampler_config.low_logits_varentropy_threshold, 'rgba(255, 165, 0, 0.2)'),
                (self.sampler_config.medium_logits_varentropy_threshold, 'rgba(165, 42, 42, 0.2)'),
                (self.sampler_config.high_logits_varentropy_threshold, 'rgba(128, 0, 128, 0.2)')
            ], 'logits'),
            ('attention_entropy', 'x', [
                (self.sampler_config.low_attention_entropy_threshold, 'rgba(255, 192, 203, 0.2)'),
                (self.sampler_config.medium_attention_entropy_threshold, 'rgba(0, 255, 255, 0.2)'),
                (self.sampler_config.high_attention_entropy_threshold, 'rgba(255, 255, 0, 0.2)')
            ], 'attention'),
            ('attention_varentropy', 'y', [
                (self.sampler_config.low_attention_varentropy_threshold, 'rgba(70, 130, 180, 0.2)'),
                (self.sampler_config.medium_attention_varentropy_threshold, 'rgba(244, 164, 96, 0.2)'),
                (self.sampler_config.high_attention_varentropy_threshold, 'rgba(50, 205, 50, 0.2)')
            ], 'attention')
        ]

        for threshold_type, axis, threshold_list, data_type in thresholds:
            for threshold, color in threshold_list:
                fig.add_trace(create_threshold_plane(threshold, axis, color, f'{threshold_type.replace("_", " ").title()} Threshold: {threshold}', data_type))

        # Create buttons for toggling views
        buttons = [
            dict(
                label='Show All',
                method='update',
                args=[{'visible': [True] * len(fig.data)}]
            ),
            dict(
                label='Hide All',
                method='update',
                args=[{'visible': [True, True] + [False] * (len(fig.data) - 2)}]
            ),
            dict(
                label='Logits Only',
                method='update',
                args=[{'visible': [True, False] + [True if i < 6 else False for i in range(len(fig.data) - 2)]}]
            ),
            dict(
                label='Attention Only',
                method='update',
                args=[{'visible': [False, True] + [True if i >= 6 else False for i in range(len(fig.data) - 2)]}]
            )
        ]

        # Update layout
        fig.update_layout(
            scene=dict(
                xaxis_title='Entropy',
                yaxis_title='Varentropy',
                zaxis_title='Token Position',
                aspectmode='manual',
                aspectratio=dict(x=1, y=1, z=0.5),
            ),
            margin=dict(l=0, r=0, b=0, t=40),
            title='',
            updatemenus=[dict(
                type="buttons",
                direction="right",
                x=0.0,
                y=1.1,
                xanchor='left',
                yanchor='top',
                pad={"r": 10, "t": 10},
                showactive=True,
                buttons=buttons
            )],
            autosize=True,
            legend=dict(x=0.02, y=0.98, xanchor='left', yanchor='top'),
        )
        '''
        # Export data to file
        export_data = {
            "tokens": [self.tokenizer.decode([token]) for token in generated_tokens],
            "logits_entropy": metrics_data['logits_entropy'],
            "logits_varentropy": metrics_data['logits_varentropy'],
            "attention_entropy": metrics_data['attention_entropy'],
            "attention_varentropy": metrics_data['attention_varentropy'],
            "thresholds": {
                "logits_entropy": {
                    "low": self.sampler_config.low_logits_entropy_threshold,
                    "medium": self.sampler_config.medium_logits_entropy_threshold,
                    "high": self.sampler_config.high_logits_entropy_threshold
                },
                "logits_varentropy": {
                    "low": self.sampler_config.low_logits_varentropy_threshold,
                    "medium": self.sampler_config.medium_logits_varentropy_threshold,
                    "high": self.sampler_config.high_logits_varentropy_threshold
                },
                "attention_entropy": {
                    "low": self.sampler_config.low_attention_entropy_threshold,
                    "medium": self.sampler_config.medium_attention_entropy_threshold,
                    "high": self.sampler_config.high_attention_entropy_threshold
                },
                "attention_varentropy": {
                    "low": self.sampler_config.low_attention_varentropy_threshold,
                    "medium": self.sampler_config.medium_attention_varentropy_threshold,
                    "high": self.sampler_config.high_attention_varentropy_threshold
                }
            }
        }

        # Generate a unique filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"entropy_data_{timestamp}.json"

        # Save the data to a file
        with open(filename, 'w') as f:
            json.dump(export_data, f, indent=2)

        print(f"Data exported to {filename}")
        '''
        return fig

    def generate(self, prompt, max_tokens=1800, debug=True):
        # Initialize lists to store metrics
        metrics_data = {
            'logits_entropy': [],
            'logits_varentropy': [],
            'attention_entropy': [],
            'attention_varentropy': []
        }
        sampler_states = []
        generated_tokens = []

        with torch.inference_mode():
            tokens = self.tokenizer.encode("<|start_header_id|>user<|end_header_id|>\n" + prompt + "<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n", bos=True, eos=False, allowed_special='all')
            tokens = torch.tensor([tokens], dtype=torch.long).to(device)
            bsz, seqlen = tokens.shape
            cur_pos = 0
            attn_mask = build_attn_mask(seqlen, cur_pos)
            freqs_cis = precompute_freqs_cis(self.model_params.head_dim, self.model_params.max_seq_len, self.model_params.rope_theta, self.model_params.use_scaled_rope)
            kvcache = KVCache.new(self.model_params.n_layers, bsz, self.model_params.max_seq_len, self.model_params.n_local_kv_heads, self.model_params.head_dim).to(device)

            logits, kvcache, scores, _ = xfmr(self.xfmr_weights, self.model_params, tokens, cur_pos, freqs_cis[:seqlen], kvcache, attn_mask=attn_mask)
            next_token, sampler_state = sample(tokens, logits, scores, self.sampler_config, self.entropix_config, generator=self.generator)

            metrics = calculate_metrics(logits, scores)
            for key in metrics_data.keys():
                if key in metrics:
                    metrics_data[key].append(metrics[key].item())
            sampler_states.append(sampler_state)

            gen_tokens = next_token
            output = self.tokenizer.decode([next_token.item()])
            generated_tokens.append(next_token.item())
            cur_pos = seqlen
            stop = torch.tensor([128001, 128008, 128009], device=device, dtype=torch.int32).to(device)

            while cur_pos < max_tokens:
                cur_pos += 1
                logits, kvcache, scores, _ = xfmr(self.xfmr_weights, self.model_params, next_token, cur_pos, freqs_cis[cur_pos:cur_pos+1], kvcache)
                next_token, sampler_state = sample(gen_tokens, logits, scores, self.sampler_config, self.entropix_config, generator=self.generator)

                metrics = calculate_metrics(logits, scores)
                for key in metrics_data.keys():
                    if key in metrics:
                        metrics_data[key].append(metrics[key].item())
                sampler_states.append(sampler_state)
                metrics_data['attention_entropy'].append(metrics['attn_entropy'].item())
                metrics_data['attention_varentropy'].append(metrics['attn_varentropy'].item())
                generated_tokens.append(next_token.item())
                gen_tokens = torch.cat((gen_tokens, next_token), dim=1)
                output += self.tokenizer.decode(next_token.tolist()[0])
                if torch.isin(next_token, stop).any():
                    break

        if debug:
            #self.debug_visualize_metrics(metrics_data)
            self.visualize_sampler_metrics(metrics_data['logits_entropy'], metrics_data['logits_varentropy'], sampler_states)
            fig = self.visualize_token_entropy_varentropy(metrics_data, generated_tokens)
            fig.show()
        return output

    def debug_visualize_metrics(self, metrics_data):
        fig, axs = plt.subplots(3, 2, figsize=(15, 15))
        fig.suptitle('Debug Visualization of Sampler Metrics', fontsize=16)

        for idx, (key, values) in enumerate(metrics_data.items()):
            if values:  # Only plot if we have data for this metric
                row = idx // 2
                col = idx % 2
                axs[row, col].plot(values)
                axs[row, col].set_title(key)
                axs[row, col].set_xlabel('Generation Step')
                axs[row, col].set_ylabel('Value')
                axs[row, col].grid(True)

        # Add entropy_attention visualization if we have both metrics
        if metrics_data['logits_entropy'] and metrics_data['attention_entropy']:
            axs[2, 0].scatter(metrics_data['logits_entropy'], metrics_data['attention_entropy'])
            axs[2, 0].set_title('entropy_attention')
            axs[2, 0].set_xlabel('Logits Entropy')
            axs[2, 0].set_ylabel('Attention Entropy')
            axs[2, 0].grid(True)

        # Add entropy_interaction_strength visualization if we have both metrics
        if metrics_data['logits_entropy'] and metrics_data['interaction_strength']:
            axs[2, 1].scatter(metrics_data['logits_entropy'], metrics_data['interaction_strength'])
            axs[2, 1].set_title('entropy_interaction_strength')
            axs[2, 1].set_xlabel('Logits Entropy')
            axs[2, 1].set_ylabel('Interaction Strength')
            axs[2, 1].grid(True)

        plt.tight_layout()
        plt.show()

    def visualize_sampler_metrics(self, entropies, varentropies, sampler_states):
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 4), height_ratios=[4, 1], sharex=True)

        # Plot entropy and varentropy
        x = range(len(entropies))
        ax1.plot(x, entropies, label='Entropy', color='blue')
        ax1.plot(x, varentropies, label='Varentropy', color='red')
        ax1.set_ylabel('Value')
        ax1.set_title('Entropy and Varentropy over Generation Steps')
        ax1.legend()
        ax1.grid(True)

        # Define colors in the same order as SamplerState
        colors = ['lightblue', 'lightgreen', 'orange', 'pink', 'purple']
        cmap = ListedColormap(colors)

        # Explicitly map each SamplerState to its corresponding index
        state_to_num = {
            SamplerState.FLOWING: 0,
            SamplerState.TREADING: 1,
            SamplerState.EXPLORING: 2,
            SamplerState.RESAMPLING: 3,
            SamplerState.ADAPTIVE: 4
        }

        # Map sampler states to numerical values
        numeric_states = [state_to_num[state] for state in sampler_states]

        # Define normalization to map each integer to a color without interpolation
        norm = BoundaryNorm(boundaries=[-0.5 + i for i in range(len(colors)+1)],
                          ncolors=cmap.N,
                          clip=True)

        # Plot color-coded sampler states
        im = ax2.imshow([numeric_states], cmap=cmap, norm=norm, aspect='auto',
                      extent=[0, len(numeric_states), 0, 1])
        ax2.set_yticks([])
        ax2.set_title('Sampler State over Generation Steps')

        mapped_colors = [colors[state_to_num[state]] for state in sampler_states]

        # Create a custom legend for sampler states
        legend_elements = [Patch(facecolor=colors[state_to_num[state]], edgecolor='black', label=state.value)
                          for state in SamplerState]
        ax2.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.15),
                  ncol=3, fancybox=True, shadow=True)

        plt.tight_layout()
        plt.show()

# Function to initialize the model (to be run once)
def initialize_model(model_size: str = "1B") -> None:
    """
    Initialize global model instance with specified size.

    Args:
        model_size: One of "1B", or "3B"
    """
    global entropix_model

    if model_size not in MODEL_CONFIGS:
        raise ValueError(f"Invalid model size. Choose from: {list(MODEL_CONFIGS.keys())}")

    print(f"Initializing {model_size} model...")
    entropix_model = EntropixModel(model_size)
    print(f"{model_size} model initialized and ready to use!")

# Function to generate text (can be used in multiple cells)
def generate_text(prompt):
    global entropix_model
    if 'entropix_model' not in globals():
        print("Model not initialized. Please run initialize_model() first.")
        return
    response = entropix_model.generate(prompt)
    # Display the response with proper newline rendering
    display(Markdown(response))

# Generate

In [ ]:
download_weights(SELECTED_MODEL_SIZE)
#load_weights(SELECTED_MODEL_SIZE)

In [ ]:
torch.cuda.empty_cache()
initialize_model(SELECTED_MODEL_SIZE)

In [ ]:
generate_text("Give quantum qubit state representation theoric description.")

In [ ]:
generate_text("Tell me a short story about a robot learning to paint.")

In [ ]:
generate_text("What is the capital city of Spain?")

In [ ]:
generate_text("Which number is larger, 9.9 or 9.11? Find result step by step")